In this notebook, I will demonstrate how to use existing SDSS software to create a desired commissioning carton and then creat FPS designs from that carton.

# Commissioning Carton Creation

If you have a set of targets with known IDs (Gaia, 2MASS, etc), a list of targets for the carton can easily be created from querying catalogdb. For this example though, I will be creating a carton of spectrophotometric standard stars in Gaia eDR3 (https://ui.adsabs.harvard.edu/abs/2021MNRAS.503.3660P/abstract), where the IDs are not currently in catalogdb.

First, I will open Table 1 from the above paper with the standard stars:

In [1]:
import pandas as pd
import numpy as np
from sdssdb.peewee.sdss5db import catalogdb

# connect to catalogdb
catalogdb.database.connect_from_parameters(user='sdss',
                                          host='localhost',
                                          port=7500)

True

In [2]:
# load Table 1
stands = pd.read_csv('stab766_supplemental_files/Table1.csv')

In [3]:
stands

,spssId,spssName,gaiaID,RA,Dec,Bmag,Vmag,spType,inV1,inV2,flag,Notes
0,1,G191-B2B,266077145295627520,76.377669,52.830674,11.444,11.792,DA0,yes,yes,0,-
1,2,GD71,3348071631670500736,88.115437,15.886239,12.792,13.032,DA1,yes,yes,0,-
2,3,GD153,3944400490365194368,194.259493,22.030385,13.110,13.399,DA1,yes,yes,0,-
3,5,EG21,4646535078125821568,47.629732,-68.601398,11.403,11.400,DA3,yes,yes,0,-
4,6,GD50,3251244858154433536,57.209484,-0.976360,13.789,14.070,DA2,yes,yes,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...
114,347,HD271759,5476400477145058816,90.172277,-66.053705,11.093,10.873,A2,yes,yes,2,Warning spectral defects (>3%)
115,348,HD271783,5284204302730217984,90.547608,-66.583021,12.539,11.955,F5,yes,yes,2,Warning spectral defects (>3%)
116,349,HIP28618,5284151216932205312,90.616156,-66.791283,12.143,12.282,B8,yes,no,-1,Non photometric - spectral defects
117,350,LTT377,5006921282807193856,10.375121,-33.626680,11.970,10.530,K9,yes,yes,1,Warning minor spectral defects (<3%)


Next, I will do a cone search in catalogdb to match the DR2 objects to eDR3. For example, I will just assume the best match is the one with the smallest angular separation.

In [4]:
#create column for DR2 IDs
stands['gaiaID_DR2'] = 9999

# radius used for cone search
r = 3/3600
for i in range(len(stands)):
    # query catalogdb around ith standard
    stand_dr2 = (catalogdb.Gaia_DR2.select(catalogdb.Gaia_DR2.ra,
                                           catalogdb.Gaia_DR2.dec,
                                           catalogdb.Gaia_DR2.source_id)
                                    .where(catalogdb.Gaia_DR2.cone_search(stands.loc[i, 'RA'],
                                                                          stands.loc[i, 'Dec'],
                                                                          r)))
    # get the ras, decs and DR2 IDs for the query
    ras, decs, dr2_ids = map(list, zip(*list(stand_dr2.tuples())))
    ras = np.array(ras)
    decs = np.array(decs)
    # calculate seperations
    seps = np.sqrt((3600 * (ras - stands.loc[i, 'RA']) * np.cos(stands.loc[i, 'Dec'] / 57.296)) ** 2 +
                   (3600 * (decs - stands.loc[i, 'Dec'])) ** 2)
    # set best match as DR2 object with minimum seperation
    stands.loc[i, 'gaiaID_DR2'] = dr2_ids[np.argmin(seps)]

Now, similar to our most common example, we have a list of known IDs that we can use to pull the relevtant information to form our carton. At this point, we will need the catalogids, RAs and DECs of the object in catalogdb. 

It will be important here to specify the version of catalogdb we will be using. You can see the catalogdb versions by:

In [5]:
ver = catalogdb.Version.select()

for v in ver:
    print(v.id, v.plan, v.tag)

21 0.1.0 0.1.0
3 0.1.0-alpha.1 0.1.0-alpha.1
16 0.1.0-alpha.5 0.1.0-beta.2
17 0.1.0-alpha.6 0.1.0-beta.3
23 0.5.0-beta.1 0.2.0b0
24 0.5.0-beta.2 0.2.0b1
999 test test
9 0.1.0-alpha.2 0.1.0-alpha.2
10 0.1.0-alpha.3 0.1.0-alpha.3
18 0.1.0-alpha.7 0.1.0-beta.3
11 0.1.0-alpha.4 0.1.0-alpha.4
25 0.5.0 0.2.2
13 0.1.0-beta.1 0.1.0b3


For this, we will be using the most up-to-date of catalogdb version 0.5.0:

In [6]:
# query catalogdb for Gaia DR2 objects in our list of standards in catalogdb version 0.5.0
stands_cdb = (catalogdb.Gaia_DR2.select(catalogdb.Gaia_DR2.ra,
                                        catalogdb.Gaia_DR2.dec,
                                        catalogdb.Gaia_DR2.source_id,
                                        catalogdb.Catalog.catalogid)
                                .join(catalogdb.TIC_v8)
                                .join(catalogdb.CatalogToTIC_v8)
                                .join(catalogdb.Catalog)
                                .join(catalogdb.Version)
                                .where((catalogdb.Gaia_DR2.source_id.in_(stands['gaiaID_DR2'].tolist())) &
                                       (catalogdb.Version.id == 25)))

# get result
ras, decs, dr2_ids, catalogids = map(list, zip(*list(stands_cdb.tuples())))

Now we can create a .fits file that will serve as our carton. This .fits file can be used by the target selection code to add this commissioning carton to targetdb. The .fits will need to have the following columns, where you just need to change the array input to what will be used for this specific carton.

In [7]:
from astropy.io import fits

# number of rows in table for creating new blank columns
nrows = len(catalogids)

# create all columns

# add the ids for the targets in the carton
col1 = fits.Column(name='id',
                   format='K',
                   array=catalogids)

# specify the id types (i.e. catalogid, Gaia ID, 2MASS id, etc)
col2 = fits.Column(name='id_type',
                   format='20A',
                   array=['catalogid'] * nrows)

# right ascension of the targets
col3 = fits.Column(name='ra',
                   format='D',
                   array=ras)

# declination of the targets
col4 = fits.Column(name='dec',
                   format='D',
                   array=decs)

# specify if fiber should be offset from target position (in arcseconds)
col5 = fits.Column(name='delta_ra',
                   format='D',
                   array=[0.] * nrows)

# specify if fiber should be offset from target position (in arcseconds)
col6 = fits.Column(name='delta_dec',
                   format='D',
                   array=[0.] * nrows)

# specify the priority of each target
col7 = fits.Column(name='priority',
                   format='J',
                   array=[1] * nrows)

# specify the value for each target
col8 = fits.Column(name='value',
                   format='J',
                   array=[1] * nrows)

# add the cadence for the observation, from list of cadences: https://wiki.sdss.org/display/OPS/Cadence+Alignment
col9 = fits.Column(name='cadence',
                   format='20A',
                   array=['bright_1x1'] * nrows)

# add the effective wavelength for observation. Defaults are 16000A for APOGEE and 5500A for BOSS
col10 = fits.Column(name='lambda_eff',
                    format='D',
                    array=[5500] * nrows)

# create column definitions for all columns
cols = fits.ColDefs([col1, col2, col3, col4, col5, col6, col7, col8, col9, col10])

# create the table HDU
hdu = fits.BinTableHDU.from_columns(cols)

# write table to .fits file
hdu.writeto('spectrophoto_standards_carton_example.fits')

# Creating Commissioning Designs

Once you have created a commissioning carton (that has then gone through target selection), you can then create a commissioning design. For this example, I will demonstrate how to create a design for a carton that is already in targetdb, specifically an all-sky design. First, we have to select all skies near some field center in targetdb.

In [8]:
from sdssdb.peewee.sdss5db import targetdb

# connect to targetdb
targetdb.database.connect_from_parameters(user='sdss',
                                          host='localhost',
                                          port=7500)

True

In [9]:
from peewee import *

# set search radius based on field size for APO or LCO
observatory = 'APO'
if observatory == 'APO':
    r_search = 1.49
else:
    r_serach = 0.95

# specify the field center and position angle
racen = 20.
deccen = 20.
position_angle=24.188576

# specify the commissioning carton
carton = 'ops_sky_boss'

# get all of the targets in the commissioning carton near the field center
boss_sky = (targetdb.Target.select(targetdb.Target.catalogid,
                                   targetdb.Target.ra,
                                   targetdb.Target.dec,
                                   targetdb.Target.pk,
                                   targetdb.CartonToTarget.priority,
                                   targetdb.CartonToTarget.value,
                                   targetdb.Cadence.label)
                    .join(targetdb.CartonToTarget)
                    .join(targetdb.Cadence, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Carton)
                    .where((targetdb.Carton.carton == carton) & 
                           (targetdb.Target.cone_search(racen, deccen, r_search))))

# grab the results
catalogid, ra, dec, target_pk, priority, value, cadences = map(list, zip(*list(boss_sky.tuples())))
catalogid = np.array(catalogid, dtype=np.int64)
ra = np.array(ra)
dec = np.array(dec)
target_pk = np.array(target_pk, dtype=np.int64)
priority = np.array(priority)
value = np.array(value)

Next, we will want to create the design using using Robostrategy. To do this, we first need to estimate the x,y coordinates of the targets in the FBS plane use coordio. To do this, we will need to also specify the observation time of the design in Julian Days.

In [10]:
from coordio.utils import radec2wokxy

# specify observation time
obsTime = 2459145.5

# convert to x,y
x, y, fieldWarn, HA, PA_coordio = radec2wokxy(ra=ra,
                                              dec=dec,
                                              coordEpoch=np.array([2457174] * len(ra)),
                                              waveName=np.array(list(map(lambda x:x.title(), ['BOSS'] * len(ra)))),
                                              raCen=racen,
                                              decCen=deccen,
                                              obsAngle=position_angle,
                                              obsSite=observatory,
                                              obsTime=obsTime)

Now we can finally build the design. This requires that we create a specially formatted array for Robostrategy to build the design. This is demonstrated below.

In [11]:
import robostrategy.field as field
import roboscheduler.cadence as cadence

# need to load cadences before building designs
cadence.CadenceList().fromdb()

# set cadence. must be in list of loaded cadences
# set cadence here because NONEs currently for sky carton
cadence = 'bright_1x1'

# create the field object
f = field.Field(racen=racen, deccen=deccen, pa=position_angle,
                    field_cadence=cadence, observatory=observatory.lower())

# set the required skies, in this case all fibers
f.required_calibrations['sky_boss'] = 500
f.required_calibrations['sky_apogee'] = 0
f.required_calibrations['standard_boss'] = 0
f.required_calibrations['standard_apogee'] = 0

# create array for RS field
N = len(ra)
# these are datatypes from robostrategy.Field
targets_dtype = np.dtype([('ra', np.float64),
                          ('dec', np.float64),
                          ('x', np.float64),
                          ('y', np.float64),
                          ('within', np.int32),
                          ('incadence', np.int32),
                          ('priority', np.int32),
                          ('value', np.float32),
                          ('program', np.unicode_, 30),
                          ('carton', np.unicode_, 30),
                          ('category', np.unicode_, 30),
                          ('cadence', np.unicode_, 30),
                          ('fiberType', np.unicode_, 10),
                          ('catalogid', np.int64),
                          ('rsid', np.int64),
                          ('target_pk', np.int64)])

# create an empty array
targs = np.zeros(N, dtype=targets_dtype)

# fill in the relevant columns
targs['ra'] = ra
targs['dec'] = dec
targs['x'] = x
targs['y'] = y
targs['within'] = np.zeros(N, dtype=np.int32) + 1
targs['incadence'] = np.zeros(N, dtype=np.int32) + 1
targs['priority'] = priority
targs['value'] = value
# for program and carton, these are both 'CALIBRATION' for all calibration targets (i.e. sky or standard)
# for science targets, program would be bhm/mwm and carton is the carton label
# don't know how to handle this for commisioning cartons yet
targs['program'] = np.array(['CALIBRATION'] * N, dtype='<U30')
targs['carton'] = np.array(['CALIBRATION'] * N, dtype='<U30')
# set category. Can either be: 'sky_apogee', 'sky_boss', 'standard_apogee', 'standard_boss' or 'science'
targs['category'] = np.array(['sky_boss'] * N, dtype='<U30')
targs['cadence'] = np.array([cadence] * N, dtype='<U30')
targs['fiberType'] = np.array(['BOSS'] * N, dtype='<U10')
targs['catalogid'] = catalogid
targs['rsid'] = np.arange(N, dtype=np.int64) + 1
targs['target_pk'] = target_pk

# assign targets
f.targets_fromarray(targs)

f.assign()

You can print out the results of the assigments for the design to see how close Robostrategy got to the desired design.

In [12]:
print(f.assess())

Field cadence: bright_1x1

Calibration targets:
 sky_boss (want 500): 386
 standard_boss (want 0): 0
 sky_apogee (want 0): 0
 standard_apogee (want 0): 0

Science targets:
 BOSS targets assigned: 0
 APOGEE targets assigned: 0
 Targets per epoch: 0

Carton completion:




# Adding Commissioning Design to targetdb

Finally, if you have created a design to your liking, you can add the design to targetdb using Mugatu. To do this, we will use the results of the above design to create a mugatu.FPSDesign object:

In [13]:
from mugatu.fpsdesign import FPSDesign

# create empty arrays for input
catalogids = np.zeros(500, dtype=np.int64) - 1
ra = np.zeros(500, dtype=float) - 9999.99
dec = np.zeros(500, dtype=float) - 9999.99
fiberID = np.zeros(500, dtype=np.int64) - 1
obsWavelength = np.zeros(500, dtype='<U10')
priority = np.zeros(500, dtype=int) - 1

# add assignments to the empty arrays
for i in range(len(f.targets)):
    if f.assignments[i][2][0] != -1:
        rid = f.assignments[i][2][0]
        catalogids[rid] = f.targets[i]['catalogid']
        ra[rid] = f.targets[i]['ra']
        dec[rid] = f.targets[i]['dec']
        fiberID[rid] = rid
        obsWavelength[rid] = f.targets[i]['fiberType']
        priority[rid] = f.targets[i]['priority']

# create a mugatu.FPSDesign object that is specified as a manual design
fps_design = FPSDesign(design_pk=-1,
                       obsTime=obsTime,
                       racen=racen,
                       deccen=deccen,
                       position_angle=position_angle,
                       observatory=observatory,
                       mode_pk=None,
                       catalogids=catalogids,
                       ra=ra,
                       dec=dec,
                       fiberID=fiberID,
                       obsWavelength=obsWavelength,
                       priority=priority,
                       design_file=None,
                       manual_design=True)

Before adding the design to targetdb, we will want to validate the design one more time to ensure that there are no collisions that Robostratgegy missed:

In [14]:
fps_design.validate_design()

[WARNING]: Some targets could not be assigned to fiber (MugatuWarning)
[WARNING]: Some targets removed from design due to collisions (MugatuWarning)


Some fibers were not able to be assigned due to fibers not being able to reach targets and due to collisions. At the creating of this notebook (4/21/2021), this is due to Robostrategy not using coordio to estimate x,y coordinates during assignemnt, while mugatu does. This will be fixed in a future release of robostratgegy, which will mitigate this issue and all designs at this point from robostrategy should be completely valid.

Finally, we can add the design to targetdb using the function below. As a note, if the field you are using is not already in targetdb, make sure that the fieldid is unique. For this example, this field has been used as other tests so a warning will come up letting you know the field is already in targetdb (and thus a new entry was not created for this field).

In [15]:
fps_design.design_to_targetdb(cadence=cadence,
                              fieldid=1,
                              targetdb_ver=49,
                              exposure=0,
                              carton=carton)

[WARNING]: Field already exists in targetdb (MugatuWarning)
